In [1]:
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from ignite.metrics import Accuracy
from tqdm.notebook import tqdm

---
## 数据部分

In [2]:
class ModelNet40(Dataset):
    def __init__(self, root, split='train', npoints=1024):
        super(ModelNet40, self).__init__()
        self.npoints = npoints
        self.class_to_idx = {}

        with open(os.path.join(root, 'modelnet40_shape_names.txt'), 'r') as f:
            for i, line in enumerate(f):
                line = line.strip()
                self.class_to_idx[line] = i

        self.file_paths = []
        self.labels = []
        with open(os.path.join(root, 'modelnet40_'+split+'.txt'), 'r') as f:
            for line in f:
                line = line.strip()
                temp = line.split('_')
                self.file_paths.append(os.path.join(root, '_'.join(temp[0:-1]), line + '.txt'))
                self.labels.append(self.class_to_idx['_'.join(temp[0:-1])])
    
    def __len__(self):
        return len(self.file_paths)

    def pcd_norm(self, points):
        mean = points.mean(axis=0)
        points = points - mean
        max_dis = (np.sqrt((np.square(points)).sum(axis=1))).max()
        points = points / max_dis

        return points

    def __getitem__(self, index):
        file = self.file_paths[index]
        label = self.labels[index]

        points = np.genfromtxt(file, delimiter=',', dtype=np.float32)

        points = points[0:self.npoints, :]

        points[:, 0:3] = self.pcd_norm(points[:, 0:3])

        return points, label

In [3]:
train_dataset = ModelNet40('modelnet40_normal_resampled', split='train')
test_dataset = ModelNet40('modelnet40_normal_resampled', split='test')

In [4]:
print(train_dataset.class_to_idx)
print(len(train_dataset))
print(len(test_dataset))

{'airplane': 0, 'bathtub': 1, 'bed': 2, 'bench': 3, 'bookshelf': 4, 'bottle': 5, 'bowl': 6, 'car': 7, 'chair': 8, 'cone': 9, 'cup': 10, 'curtain': 11, 'desk': 12, 'door': 13, 'dresser': 14, 'flower_pot': 15, 'glass_box': 16, 'guitar': 17, 'keyboard': 18, 'lamp': 19, 'laptop': 20, 'mantel': 21, 'monitor': 22, 'night_stand': 23, 'person': 24, 'piano': 25, 'plant': 26, 'radio': 27, 'range_hood': 28, 'sink': 29, 'sofa': 30, 'stairs': 31, 'stool': 32, 'table': 33, 'tent': 34, 'toilet': 35, 'tv_stand': 36, 'vase': 37, 'wardrobe': 38, 'xbox': 39}
9843
2468


In [5]:
pcd, gt = train_dataset[0]

In [6]:
# 检验norm是否正确
mean = pcd.mean(axis=0)
print(mean)
dis = np.sqrt((np.square(pcd[:, 0:3])).sum(axis=1))
print(dis)
max_dis = dis.max()
print(max_dis)

[ 5.6534191e-09 -3.8970029e-08 -2.0793777e-08 -2.2611087e-02
  6.2716450e-03  6.5562748e-03]
[0.18740387 0.9741179  0.99170816 ... 0.42549008 0.17417295 0.16141398]
1.0


In [7]:
print(type(pcd), type(gt))
print(pcd.shape, pcd.dtype)

<class 'numpy.ndarray'> <class 'int'>
(1024, 6) float32


In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=24, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=24, shuffle=False, num_workers=8)

In [9]:
pcds, labels = next(iter(train_dataloader))

In [10]:
print(pcds.shape, pcds.dtype)
print(labels.shape, labels.dtype)

torch.Size([24, 1024, 6]) torch.float32
torch.Size([24]) torch.int64


---
## 模型部分

In [11]:
class PointNet(nn.Module):
    def __init__(self, class_num):
        super(PointNet, self).__init__()
        self.mlp_1 = nn.Sequential(nn.Conv1d(6, 64, kernel_size=1),
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(inplace=True),
                                    nn.Conv1d(64, 64, kernel_size=1),
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(inplace=True),
                                    nn.Conv1d(64, 128, kernel_size=1),
                                    nn.BatchNorm1d(128),
                                    nn.ReLU(inplace=True),
                                    nn.Conv1d(128, 1024, kernel_size=1),
                                    nn.BatchNorm1d(1024),
                                    nn.ReLU(inplace=True))
        self.mlp_2 = nn.Sequential(nn.Linear(1024, 512),
                                    nn.BatchNorm1d(512),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(512, 256),
                                    nn.Dropout(0.4),
                                    nn.BatchNorm1d(256),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(256, class_num))
    
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.mlp_1(x)
        x, _ = x.max(dim=2)
        y = self.mlp_2(x)

        return y

In [12]:
def train_loop(dataloader, model, loss_fn, metric_fn, optimizer, device, cur_epoch, total_epoch, show_gap):
    model.train()
    if cur_epoch % show_gap == 0:
        pbar = tqdm(dataloader, desc=f'Epoch {cur_epoch}/{total_epoch}', unit='batch')
    else:
        pbar = dataloader

    for x, y in pbar:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        metric_fn.reset()
        metric_fn.update((y_pred, y))
        acc = metric_fn.compute()

        if cur_epoch % show_gap == 0:
            pbar.set_postfix_str(f'loss={loss:.4f}, acc={acc:.4f}')

In [13]:
def test_loop(dataloader, model, loss_fn, metric_fn, device, show_gap):
    model.eval()
    steps = len(dataloader)
    loss = 0
    acc = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss += loss_fn(y_pred, y)

            metric_fn.reset()
            metric_fn.update((y_pred, y))
            acc += metric_fn.compute()
    loss = loss / steps
    acc = acc / steps

    print(f'test_loss={loss:.4f}, test_acc={acc:.4f}')

---
## 开始训练

In [14]:
device = 'cuda:0'

pointnet = PointNet(40).to(device)
loss_fn = nn.CrossEntropyLoss()
metric_fn = Accuracy(device=device)
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001, weight_decay=1e-4)

In [ ]:
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 50)

epochs = 50
show_gap = 1
for i in range(epochs):
    train_loop(train_dataloader, pointnet, loss_fn, metric_fn, optimizer, device, i, epochs, show_gap)
    test_loop(test_dataloader, pointnet, loss_fn, metric_fn, device, show_gap)
    lr_scheduler.step()

In [ ]:
torch.save(pointnet.state_dict(), 'pointnet_cls.pth')